# Merge.csv

Tylko energia grupowana po regionie i czasie. LSTM, Dense, XGB

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import xgboost as xgb
warnings.filterwarnings('ignore')

2023-10-15 02:49:19.846404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 02:49:20.966249: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../../data/merge.csv')
df.head()

,network_code,receiver_code,receiver_type,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,p_status,p_weight,p_travel_sec,...,source_magnitude_author,source_mechanism_strike_dip_rake,source_distance_deg,source_distance_km,back_azimuth_deg,snr_db,coda_end_sample,trace_start_time,trace_category,trace_name
0,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-21 05:55:00,noise,109C.TA_201510210555_NO
1,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-06 14:50:00,noise,109C.TA_201511061450_NO
2,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-07 02:20:00,noise,109C.TA_201511070220_NO
3,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-14 05:15:00,noise,109C.TA_201511140515_NO
4,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-25 18:50:00,noise,109C.TA_201512251850_NO


In [3]:
df = df[df["trace_category"] == 'earthquake_local']
df.rename(columns={"source_latitude": "latitude", "source_longitude": "longitude", "source_magnitude": "mag", "trace_start_time": "time"}, inplace=True)

In [4]:
df = df[["time", "latitude", "longitude", "mag"]]

In [5]:
df["time"] = pd.to_datetime(df["time"], format="mixed")
df

,time,latitude,longitude,mag
235426,2006-07-23 15:59:00.960,33.74960,-117.49380,3.60
235427,2006-11-03 15:56:53.610,32.70770,-116.04460,4.30
235428,2006-11-03 16:12:24.700,32.72530,-116.03480,3.60
235429,2006-11-14 13:32:22.540,32.70630,-116.02410,3.80
235430,2006-11-27 10:46:41.060,31.96790,-117.19440,3.60
...,...,...,...,...
1265652,2017-06-21 09:40:43.810,44.77950,-111.03833,0.39
1265653,2017-06-21 12:21:05.390,44.76967,-110.99700,1.72
1265654,2017-06-21 12:23:40.200,44.77100,-110.99900,1.61
1265655,2017-06-21 13:34:40.330,44.77567,-111.03983,1.25


In [6]:
time_cut = dt.datetime(1973, 1, 1)
df = df[df["time"] > time_cut]
df["energy"] = 10**(1.44*df["mag"]+5.24)
df["label"] = np.where(df["mag"] > 5, 1, 0)

In [7]:
# keep only year and month in time
df["time"] = df["time"].dt.strftime('%Y-%m')

In [8]:
geo_split = 1

In [9]:
df["latitude"] = (df["latitude"] // geo_split).astype(int)
df["latitude"] = df["latitude"] + np.abs(np.min(df["latitude"]))

df["longitude"] = (df["longitude"] // geo_split).astype(int)
df["longitude"] = df["longitude"] + np.abs(np.min(df["longitude"]))
df["pos"] = df["latitude"].astype(str) + "_" + df["longitude"].astype(str)

In [10]:
def filter_regions(df: pd.DataFrame, threshold: int) -> pd.DataFrame:
    tmp = []
    df_f = df[df["mag"] >= 5]
    df_agg = df_f.groupby(["pos"])["mag"].count().reset_index()
    for _, row in df_agg.iterrows():
        if row["mag"] >= threshold:
            tmp.append(row["pos"])
    df = df[df["pos"].isin(tmp)]
    return df

In [11]:
th = 4
df = filter_regions(df, th)

In [12]:
df

,time,latitude,longitude,mag,energy,label,pos
235426,2006-07,77,62,3.6,2.654606e+10,0,77_62
235431,2006-11,76,64,3.8,5.152286e+10,0,76_64
235432,2006-11,76,64,4.1,1.393157e+11,0,76_64
235433,2006-11,76,64,3.6,2.654606e+10,0,76_64
235434,2006-11,76,64,3.9,7.177943e+10,0,76_64
...,...,...,...,...,...,...,...
1265360,2015-01,88,65,3.2,7.046931e+09,0,88_65
1265361,2015-01,88,65,3.2,7.046931e+09,0,88_65
1265364,2015-01,88,65,2.8,1.870682e+09,0,88_65
1265369,2015-01,88,65,3.4,1.367729e+10,0,88_65


In [13]:
df_agg = df.groupby(["latitude", "longitude", "time"]).agg({"energy": "sum", "label": "max"}).reset_index()
df_agg

,latitude,longitude,time,energy,label
0,1,353,2004-10,5.248075e+11,0
1,1,353,2011-02,1.030457e+11,0
2,1,353,2013-08,1.030457e+11,0
3,1,353,2014-02,2.703958e+11,0
4,1,353,2014-10,3.630781e+09,0
...,...,...,...,...,...
7813,113,35,2018-08,4.093718e+14,1
7814,113,35,2018-09,5.342576e+11,0
7815,113,35,2018-10,1.808764e+12,0
7816,113,35,2018-11,1.097306e+12,0


In [14]:
def make_ds(df_agg: pd.DataFrame, block_size: int) -> (pd.DataFrame, pd.DataFrame):
    dfs_train, dfs_val = [], []
    for i in df_agg["latitude"].unique():
        for j in df_agg["longitude"].unique():
            tmp = df_agg[(df_agg["latitude"] == i) & (df_agg["longitude"] == j)]
            if len(tmp) > 1:
                tmp = tmp.set_index("time")
                idx = pd.date_range(tmp.index.min(), df_agg["time"].max(), freq="MS").strftime('%Y-%m')
                tmp = tmp.reindex(idx, fill_value=0)
                tmp.index.name = "time"
                tmp["label"] = tmp["label"].shift(-1)
                tmp["latitude"] = i
                tmp["longitude"] = j
                n = int(0.8 * len(tmp))
                df_train = tmp[:n]
                df_val = tmp[n:]
                scaler = MinMaxScaler()
                df_train["energy"] = scaler.fit_transform(df_train[["energy"]])
                df_val["energy"] = scaler.transform(df_val[["energy"]])
                for idx in range(1, block_size):
                    df_train["energy" + str(idx)] = df_train["energy"].shift(idx)
                    df_val["energy" + str(idx)] = df_val["energy"].shift(idx)
                dfs_train.append(df_train)
                dfs_val.append(df_val)
    df_final_train = pd.concat(dfs_train)
    df_final_val = pd.concat(dfs_val)
    return df_final_train, df_final_val

In [15]:
block_size = 16
df_train, df_val = make_ds(df_agg, block_size)
df_train

,latitude,longitude,energy,label,energy1,energy2,energy3,energy4,energy5,energy6,energy7,energy8,energy9,energy10,energy11,energy12,energy13,energy14,energy15
time,,,,,,,,,,,,,,,,,,,
2004-10,1,353,0.001150,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-11,1,353,0.000000,0.0,0.001150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-12,1,353,0.000000,0.0,0.000000,0.001150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01,1,353,0.000000,0.0,0.000000,0.000000,0.001150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-02,1,353,0.000000,0.0,0.000000,0.000000,0.000000,0.001150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-06,113,35,0.000072,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.007306,0.000000,0.000000,0.017806,0.000000,0.000000,0.000000,0.000000
2017-07,113,35,0.000143,0.0,0.000072,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.007306,0.000000,0.000000,0.017806,0.000000,0.000000,0.000000
2017-08,113,35,0.002004,0.0,0.000143,0.000072,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.007306,0.000000,0.000000,0.017806,0.000000,0.000000


In [16]:
df_train.dropna(inplace=True)
df_val.dropna(inplace=True)

In [23]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True), input_shape=(block_size, 1)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2),
])

In [24]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [25]:
x_train = df_train.drop(["label", "latitude", "longitude"], axis=1).to_numpy()
x_train = x_train.reshape(-1, block_size, 1)
y_train = df_train["label"].to_numpy().reshape(-1, 1)
x_val = df_val.drop(["label", "latitude", "longitude"], axis=1).to_numpy()
x_val = x_val.reshape(-1, block_size, 1)
y_val = df_val["label"].to_numpy().reshape(-1, 1)

In [26]:
# calculate balance of classes
unique, counts = np.unique(y_train, return_counts=True)
counts[0] / len(y_train)

0.9861290786258444

In [27]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [28]:
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_val, y_val), shuffle=True)

Epoch 1/20


435/435 [==============================] - 47s 76ms/step - loss: 0.0946 - accuracy: 0.9854 - f1_score: 0.4963 - val_loss: 0.0562 - val_accuracy: 0.9907 - val_f1_score: 0.4977
Epoch 2/20
435/435 [==============================] - 30s 70ms/step - loss: 0.0760 - accuracy: 0.9861 - f1_score: 0.4965 - val_loss: 0.0535 - val_accuracy: 0.9907 - val_f1_score: 0.4977
Epoch 3/20
435/435 [==============================] - 30s 69ms/step - loss: 0.0741 - accuracy: 0.9861 - f1_score: 0.4965 - val_loss: 0.0536 - val_accuracy: 0.9907 - val_f1_score: 0.4977
Epoch 4/20
435/435 [==============================] - 32s 74ms/step - loss: 0.0737 - accuracy: 0.9861 - f1_score: 0.4965 - val_loss: 0.0534 - val_accuracy: 0.9907 - val_f1_score: 0.4977
Epoch 5/20
435/435 [==============================] - 36s 83ms/step - loss: 0.0742 - accuracy: 0.9861 - f1_score: 0.4965 - val_loss: 0.0564 - val_accuracy: 0.9907 - val_f1_score: 0.4977
Epoch 6/20
435/435 [==============================] - 39s 88ms/step - loss: 0.074

KeyboardInterrupt: 

In [75]:
x_train = df_train.drop(["time", "label", "latitude", "longitude"], axis=1).to_numpy()
y_train = df_train["label"].to_numpy().reshape(-1, 1)
x_val = df_val.drop(["time", "label", "latitude", "longitude"], axis=1).to_numpy()
y_val = df_val["label"].to_numpy().reshape(-1, 1)

In [79]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [80]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(block_size, activation='relu', input_shape=(block_size, )),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2),
])

In [81]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [84]:
model.fit(x_train, y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

Epoch 1/20
329/329 [==============================] - 6s 18ms/step - loss: 0.2928 - accuracy: 0.8852 - f1_score: 0.4703 - val_loss: 0.3059 - val_accuracy: 0.8797 - val_f1_score: 0.4680
Epoch 2/20
329/329 [==============================] - 5s 16ms/step - loss: 0.2912 - accuracy: 0.8851 - f1_score: 0.4703 - val_loss: 0.3076 - val_accuracy: 0.8797 - val_f1_score: 0.4680
Epoch 3/20
329/329 [==============================] - 5s 16ms/step - loss: 0.2921 - accuracy: 0.8851 - f1_score: 0.4701 - val_loss: 0.3059 - val_accuracy: 0.8797 - val_f1_score: 0.4680
Epoch 4/20
329/329 [==============================] - 5s 16ms/step - loss: 0.2911 - accuracy: 0.8852 - f1_score: 0.4703 - val_loss: 0.3091 - val_accuracy: 0.8796 - val_f1_score: 0.4680
Epoch 5/20
329/329 [==============================] - 5s 15ms/step - loss: 0.2919 - accuracy: 0.8852 - f1_score: 0.4707 - val_loss: 0.3126 - val_accuracy: 0.8797 - val_f1_score: 0.4680
Epoch 6/20
329/329 [==============================] - 5s 15ms/step - loss: 

In [86]:
x_train = df_train.drop(["time", "label", "latitude", "longitude"], axis=1).to_numpy()
y_train = df_train["label"].to_numpy().reshape(-1, 1)
x_val = df_val.drop(["time", "label", "latitude", "longitude"], axis=1).to_numpy()
y_val = df_val["label"].to_numpy().reshape(-1, 1)

In [87]:
model = xgb.XGBClassifier(n_estimators=1000, max_depth=10, learning_rate=0.01, n_jobs=-1)

In [88]:
model.fit(x_train, y_train, eval_set=[(x_val, y_val)], eval_metric='logloss', early_stopping_rounds=10)

[0]	validation_0-logloss:0.68654
[1]	validation_0-logloss:0.68007
[2]	validation_0-logloss:0.67372
[3]	validation_0-logloss:0.66750
[4]	validation_0-logloss:0.66138
[5]	validation_0-logloss:0.65537
[6]	validation_0-logloss:0.64949
[7]	validation_0-logloss:0.64371
[8]	validation_0-logloss:0.63804
[9]	validation_0-logloss:0.63247
[10]	validation_0-logloss:0.62700
[11]	validation_0-logloss:0.62163
[12]	validation_0-logloss:0.61636
[13]	validation_0-logloss:0.61116
[14]	validation_0-logloss:0.60607
[15]	validation_0-logloss:0.60106
[16]	validation_0-logloss:0.59615
[17]	validation_0-logloss:0.59132
[18]	validation_0-logloss:0.58656
[19]	validation_0-logloss:0.58189
[20]	validation_0-logloss:0.57731
[21]	validation_0-logloss:0.57280
[22]	validation_0-logloss:0.56836
[23]	validation_0-logloss:0.56399
[24]	validation_0-logloss:0.55970
[25]	validation_0-logloss:0.55548
[26]	validation_0-logloss:0.55133
[27]	validation_0-logloss:0.54724
[28]	validation_0-logloss:0.54321
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [90]:
y_pred = model.predict(x_val)

In [91]:
accuracy_score(y_val, y_pred), f1_score(y_val, y_pred)

(0.8938714841070203, 0.4426564188783475)